In [26]:
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup
import pymongo
import time
import json
import re


In [34]:
def detect_valid_string(input_string):
    """
    Detects if a string contains only letters (a-z, A-Z), periods (.), commas (,), and hyphens (-).

    Args:
        input_string: The string to check.

    Returns:
        True if the string is valid, False otherwise.
    """

    if not isinstance(input_string, str):
        return False #if input is not a string, return false.

    pattern = r"^[a-zA-Z.,\- ]+$"
    match = re.match(pattern, input_string)
    return bool(match)
#


In [15]:
def custom_query(get_url):
    #print(get_url)
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36", # A common Chrome User-Agent
        "Content-Type": "application/json", #if needed
    }
    try:
        r = requests.get(get_url, headers=headers)
        print(f"Status Code: {r.status_code}")
        print(f"Content-Type: {r.headers.get('Content-Type')}")
        print(f"Response Text:\n{r.text}")
        return r.json()
    except Exception as e:
       print(e)
       return {'error': e}

In [24]:
def scrape_text_data(url):
    try:
        # Set up Chrome WebDriver
        service = Service('../../chromedriver') # or geckodriver for firefox
        options = webdriver.ChromeOptions()
        #options.add_argument('headless')
        options.add_argument("disable-gpu")
        options.add_argument("--window-size=0,0")
        options.add_experimental_option("detach", True)

        driver = webdriver.Chrome(service=service, options=options) # or webdriver.Firefox(service=service)

        # Navigate to the webpage
        driver.get(url)
        # driver.minimize_window()
        # driver.set_window_size(1, 1)

        # Wait for JavaScript to load (adjust time as needed)
        driver.implicitly_wait(10)  # Waits up to 10 seconds for elements to appear

        # Get the rendered HTML
        html = driver.page_source

        # Close the browser
        driver.quit()

        # Parse the HTML with BeautifulSoup
        soup = BeautifulSoup(html, "html.parser")
        #print(html)
        target_element = soup.find('pre')
        #print(target_element.text)
        if target_element:
            try:
                data = json.loads(target_element.text)
                #print(data)
                return data
            except json.JSONDecodeError as e:
                print(f"JSON Decode Error: {e}")
                return {'error': e}
    except Exception as e:
       print(e)
       return {'error': e}
#test
raw_itviec = scrape_text_data('https://itviec.com/api/v1/tags/populars')

In [ ]:
#list of companies
company_list = []
for item in raw_itviec:
    if item['category'] == 'Company':
        #filter not vietnamese companies
        if not detect_valid_string(item['name']):
            print('Invalid name: ' + item['name'])  #todo need to add more signs
        else:
            company_list.append(item)
#
#print(company_list)

Invalid name: Monk's Hill Ventures (MHV)
Invalid name: Tập Đoàn Đại Trường Phát
Invalid name: Laser – AD Media
Invalid name: DM&C
Invalid name: VIỆN CÔNG NGHỆ THÔNG TIN T3H
Invalid name: Việt Tinh Anh
Invalid name: Khai Phú Investments and Migration
Invalid name: Nhựa Đông Á - DAG
Invalid name: Giao Hàng Tiết Kiệm
Invalid name: Đồng Bát Software Services and Solutions
Invalid name: PIZZA 4P'S
Invalid name: Công ty CP Công trình Viettel - Tập đoàn Viễn Thông Quân Đội (Viettel)
Invalid name: Li & Fung (Vietnam) Limited
Invalid name: PHÚ HOÀNG LAND
Invalid name: F88
Invalid name: Tâm Anh Hospital
Invalid name: TBWA\
Invalid name: Công ty cổ phần PayPay
Invalid name: Vân Tay Media
Invalid name: Dự toán ETA
Invalid name: CÔNG TY CỔ PHẦN OPEN HEALTHCARE VIỆT NAM
Invalid name: Nam Việt Media
Invalid name: 24hPay
Invalid name: Hà Đô - 756 Sài Gòn
Invalid name: Confluence Vietnam (HCMC)
Invalid name: SS4U Express
Invalid name: Công Ty Cổ Phần Đầu Tư Phát Triển Sách Và Học Liệu Điện Tử Việt Nam